## RAGAS Evaluation


documentation: https://docs.ragas.io/en/stable/getstarted/index.html

#### Standard way

it gives Attribute Error LlamaForCausalLM' object has no attribute 'set_run_config'  during evaluation

Tested also using LangchainLLMWrapper. No good results

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from langchain_community.llms import HuggingFacePipeline
from langchain_community.embeddings import HuggingFaceEmbeddings

# embedding model
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2") #they used "BAAI/bge-m3"

# evaluator
model_id = "swap-uniba/LLaMAntino-3-ANITA-8B-Inst-DPO-ITA"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)



c:\app\BaseToolkit\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 4/4 [02:35<00:00, 38.97s/it]


In [3]:
data_samples = {
    'question': ['When was the first super bowl?', 'Who won the most super bowls?'],
    'answer': ['The first superbowl was held on Jan 15, 1967', 'The most super bowls have been won by The New England Patriots'],
    'contexts' : [['The First AFL–NFL World Championship Game was an American football game played on January 15, 1967, at the Los Angeles Memorial Coliseum in Los Angeles,'], 
                    ['The Green Bay Packers...Green Bay, Wisconsin.','The Packers compete...Football Conference']],
    'ground_truth': ['The first superbowl was held on January 15, 1967', 'The New England Patriots have won the Super Bowl a record six times']
}

In [4]:
from datasets import Dataset 

dataset = Dataset.from_dict(data_samples)

In [5]:
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
)

metrics=[
        context_precision,
        faithfulness,
        answer_relevancy,
        context_recall,
    ]

#metrics = [
 #   BLEU(), ROUGE(), METEOR(), BertScore(), NIST(), CIDEr(), TER(), Spice() ]

In [10]:
from ragas import evaluate
score = evaluate(dataset, metrics=metrics, llm=model, embeddings=embedding_model)

AttributeError: 'LlamaForCausalLM' object has no attribute 'set_run_config'

In [ ]:
score

### Improving standard way using LangchainLLMWrapper and LangchainEmbeddingsWrapper


In [1]:
# attach to the existing event loop when using jupyter notebooks
import nest_asyncio

nest_asyncio.apply()

In [2]:
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
from langchain import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
)
from ragas import evaluate
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from datasets import Dataset


model_name = "microsoft/Phi-3-mini-4k-instruct"#"swap-uniba/LLaMAntino-3-ANITA-8B-Inst-DPO-ITA"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)


pipe = pipeline(
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,
    task='text-generation',
    temperature=0.1, 
    repetition_penalty=1.1,
    max_new_tokens=2000
)


langchain_llm = HuggingFacePipeline(pipeline=pipe)


embedding_model = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")

# Wrap the LLM and embeddings with the RAGAS wrappers
langchain_llm = LangchainLLMWrapper(langchain_llm)
langchain_embeddings = LangchainEmbeddingsWrapper(embedding_model)


data_samples = {
    'question': ['When was the first super bowl?', 'Who won the most super bowls?'],
    'answer': ['The first superbowl was held on Jan 15, 1967', 'The most super bowls have been won by The New England Patriots'],
    'contexts': [['The First AFL–NFL World Championship Game was an American football game played on January 15, 1967, at the Los Angeles Memorial Coliseum in Los Angeles,'],
                 ['The Green Bay Packers...Green Bay, Wisconsin.', 'The Packers compete...Football Conference']],
    'ground_truth': ['The first superbowl was held on January 15, 1967', 'The New England Patriots have won the Super Bowl a record six times']
}


dataset = Dataset.from_dict(data_samples)



c:\app\BaseToolkit\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 2/2 [01:09<00:00, 34.82s/it]
c:\app\BaseToolkit\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  warn_deprecated(
c:\app\BaseToolkit\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `Hu

In [2]:
data_samples = {
    'question': ['When was the first super bowl?', 'Who won the most super bowls?'],
    'answer': ['The first superbowl was held on Jan 15, 1967', 'The most super bowls have been won by The New England Patriots'],
    'contexts': [['The First AFL–NFL World Championship Game was an American football game played on January 15, 1967, at the Los Angeles Memorial Coliseum in Los Angeles,'],
                 ['The Green Bay Packers...Green Bay, Wisconsin.', 'The Packers compete...Football Conference']],
    'ground_truth': ['The first superbowl was held on January 15, 1967', 'The New England Patriots have won the Super Bowl a record six times']
}


dataset = Dataset.from_dict(data_samples)

In [3]:
import ragas
run_config = ragas.RunConfig(timeout=10, max_retries=1, max_wait=10)

In [4]:
result = evaluate(
    dataset=dataset,
    llm=langchain_llm,
    embeddings=langchain_embeddings,
    metrics=[
        context_precision#,
        #faithfulness#,
        #answer_relevancy#,
        #context_recall,
    ],
    raise_exceptions=False,
    run_config= run_config # Ensure exceptions are raised directly
    #is_async = True  
)


Evaluating:   0%|          | 0/2 [00:00<?, ?it/s]

c:\app\BaseToolkit\lib\site-packages\transformers\generation\configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
You are not running the flash-attention implementation, expect numerical differences.
You are not running the flash-attention implementation, expect numerical differences.
Runner in Executor raised an exception
Traceback (most recent call last):
  File "c:\app\BaseToolkit\lib\asyncio\tasks.py", line 256, in __step
    result = coro.send(None)
  File "c:\app\BaseToolkit\lib\site-packages\ragas\metrics\_context_precision.py", line 161, in _ascore
    results = await self.llm.generate(
  File "c:\app\BaseToolkit\lib\site-packages\ragas\llms\base.py", line 93, in generate
    return await agenerate_text_with_retry(
  File "c:\app\BaseToolkit\lib\site-packages\tenacity\_asyncio.py", line 88,

In [5]:
result

{'context_precision': nan}

#### Ollama case (no Lllamantino)


In [1]:
# attach to the existing event loop when using jupyter notebooks
import nest_asyncio

nest_asyncio.apply()

In [2]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.chat_models import ChatOllama

In [3]:
langchain_embeddings = OllamaEmbeddings(model = "llama3")
langchain_llm = ChatOllama(model = "llama3")

In [4]:
from datasets import Dataset 
data_samples = {
    'question': ['When was the first super bowl?'],#, 'Who won the most super bowls?'],
    'answer': ['The first superbowl was held on Jan 15, 1967'],# 'The most super bowls have been won by The New England Patriots'],
    'contexts' : [['The First AFL–NFL World Championship Game was an American football game played on January 15, 1967, at the Los Angeles Memorial Coliseum in Los Angeles,']], 
    #['The Green Bay Packers...Green Bay, Wisconsin.','The Packers compete...Football Conference']],
    'ground_truth': ['The first superbowl was held on January 15, 1967']#, 'The New England Patriots have won the Super Bowl a record six times']
}
dataset = Dataset.from_dict(data_samples)

c:\app\BaseToolkit\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
)

metrics=[
        context_precision,
        faithfulness,
        answer_relevancy,
        context_recall,
    ]

In [6]:
from ragas import evaluate

In [20]:
import asyncio
from ragas.llms.base import RunConfig
result = await asyncio.wait_for(
                    evaluate(dataset, metrics=metrics, llm = langchain_llm, embeddings = langchain_embeddings, run_config=RunConfig(thread_timeout=60)),
                    timeout=600  # 10 minutes timeout 
                    )

Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/4 [01:00<?, ?it/s]
Exception in thread Thread-37:
Traceback (most recent call last):
  File "c:\app\BaseToolkit\lib\asyncio\tasks.py", line 258, in __step
    result = coro.throw(exc)
  File "c:\app\BaseToolkit\lib\site-packages\ragas\metrics\_context_recall.py", line 169, in _ascore
    results = await self.llm.generate(
  File "c:\app\BaseToolkit\lib\site-packages\ragas\llms\base.py", line 93, in generate
    return await agenerate_text_with_retry(
  File "c:\app\BaseToolkit\lib\site-packages\tenacity\_asyncio.py", line 88, in async_wrapped
    return await fn(*args, **kwargs)
  File "c:\app\BaseToolkit\lib\site-packages\tenacity\_asyncio.py", line 57, in __call__
    await self.sleep(do)
  File "c:\app\BaseToolkit\lib\asyncio\tasks.py", line 652, in sleep
    return await future
  File "c:\app\BaseToolkit\lib\asyncio\futures.py", line 284, in __await__
    yield self  # This tells Task to wait for completion.
  File "c:\app\BaseToolkit\lib\asyncio\task

ExceptionInRunner: The runner thread which was running the jobs raised an exeception. Read the traceback above to debug it. You can also pass `raise_exceptions=False` incase you want to show only a warning message instead.

In [7]:
result = evaluate(dataset, metrics=metrics, llm = langchain_llm, embeddings = langchain_embeddings)

Evaluating:   0%|          | 0/4 [01:20<?, ?it/s]


TimeoutError: 

In [17]:
result

NameError: name 'result' is not defined

## from webinar

In [1]:
# attach to the existing event loop when using jupyter notebooks
import nest_asyncio

nest_asyncio.apply()

In [4]:
import os, sys, pprint
import pandas as pd
import numpy as np
import ragas, datasets

# Libraries to customize ragas critic model.
from ragas.llms import LangchainLLMWrapper
from langchain_community.chat_models import ChatOllama

# Libraries to customize ragas embedding model.
from langchain_huggingface import HuggingFaceEmbeddings
from ragas.embeddings import LangchainEmbeddingsWrapper

# Import custom functions for evaluation.
#sys.path.append("../../Evaluation")
#import eval_ragas as _eval_ragas

# Import the evaluation metrics.
from ragas.metrics import (
    context_recall, 
    context_precision, 
    faithfulness, 
    answer_relevancy, 
    answer_similarity,
    answer_correctness
    )
'''ArithmeticError
cwd = os.getcwd()
relative_path = '/../../Evaluation/data/blog_eval_answers.csv'
file_path = cwd + relative_path
# print(f"file_path: {file_path}")

# Read ground truth answers from a CSV file.
eval_df = pd.read_csv(file_path, header=0, skip_blank_lines=True)
display(eval_df.head())
'''

'ArithmeticError\ncwd = os.getcwd()\nrelative_path = \'/../../Evaluation/data/blog_eval_answers.csv\'\nfile_path = cwd + relative_path\n# print(f"file_path: {file_path}")\n\n# Read ground truth answers from a CSV file.\neval_df = pd.read_csv(file_path, header=0, skip_blank_lines=True)\ndisplay(eval_df.head())\n'

In [2]:
from datasets import Dataset 
data_samples = {
    'question': ['When was the first super bowl?', 'Who won the most super bowls?'],
    'answer': ['The first superbowl was held on Jan 15, 1967', 'The most super bowls have been won by The New England Patriots'],
    'contexts': [['The First AFL–NFL World Championship Game was an American football game played on January 15, 1967, at the Los Angeles Memorial Coliseum in Los Angeles,'],
                 ['The Green Bay Packers...Green Bay, Wisconsin.', 'The Packers compete...Football Conference']],
    'ground_truth': ['The first superbowl was held on January 15, 1967', 'The New England Patriots have won the Super Bowl a record six times']
}


dataset = Dataset.from_dict(data_samples)

c:\app\BaseToolkit\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
%pip install -qU langchain-ollama

Note: you may need to restart the kernel to use updated packages.


In [5]:
##########################################
# Set the evaluation type.
EVALUATE_WHAT = 'ANSWERS' 
EVALUATE_WHAT = 'CONTEXTS'
##########################################

# Set the metrics to evaluate.
if EVALUATE_WHAT == 'ANSWERS':
    eval_metrics=[
        answer_relevancy,
        answer_similarity,
        answer_correctness,
        faithfulness,
        ]
    metrics = ['answer_relevancy', 'answer_similarity', 'answer_correctness', 'faithfulness']
elif EVALUATE_WHAT == 'CONTEXTS':
    eval_metrics=[
        context_recall, 
        context_precision, 
        ]
    metrics = ['context_recall', 'context_precision']
    
# Change the default llm-as-critic model to gpt-3.5-turbo.
# LLM_NAME = "gpt-3.5-turbo" #OpenAI
# ragas_llm = ragas.llms.llm_factory(model=LLM_NAME)

# Change the default the llm-as-critic model to local llama3.
LLM_NAME = 'llama3' #'llama3.1'
ragas_llm = LangchainLLMWrapper(langchain_llm=ChatOllama(model=LLM_NAME))

# Change the default embeddings models to use model on HuggingFace.
EMB_NAME = "BAAI/bge-large-en-v1.5"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}
lc_embed_model = HuggingFaceEmbeddings(
    model_name=EMB_NAME,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)
ragas_emb = LangchainEmbeddingsWrapper(embeddings=lc_embed_model)

# Change embeddings and critic models for each metric.
for metric in metrics:
    globals()[metric].llm = ragas_llm
    globals()[metric].embeddings = ragas_emb



In [6]:
# 2. Define function to evaluate RAGAS model.
def evaluate_ragas_model(data_samples, 
                         ragas_eval_metrics, 
                         what_to_evaluate='CONTEXTS'):
    """Evaluate the RAGAS model using the input pandas df."""

    #temp_df = pandas_eval_df.copy()
    ragas_results_df_list = []
    #scores = []
    # Set the metrics to evaluate.
    if EVALUATE_WHAT == 'ANSWERS':
        eval_metrics=[
            answer_relevancy,
            answer_similarity,
            answer_correctness,
            faithfulness,
            ]
        #metrics = ['answer_relevancy', 'answer_similarity', 'answer_correctness', 'faithfulness']
    elif EVALUATE_WHAT == 'CONTEXTS':
            eval_metrics=[
            context_recall, 
            #context_precision,
            ]
        #metrics = ['context_recall', 'context_precision']

    ragas_results = ragas.evaluate(data_samples, metrics=eval_metrics)
    print(ragas_results)

    '''
    # Return evaluations as pandas df.
    temp = ragas_results.to_pandas()

    temp_score = -1.0
    if what_to_evaluate == "CONTEXTS":
        # Calculate context F1 scores.
        temp['context_f1'] = \
                2.0 * temp.context_precision * temp.context_recall \
                / (temp.context_precision + temp.context_recall)
        temp = temp.fillna(0.0)
        # Calculate Retrieval average score.
        avg_retrieval_f1 = np.round(temp.context_f1.mean(),2)
        temp_score = avg_retrieval_f1

    elif what_to_evaluate == "ANSWERS":
        # Calculate avg LLM answer scores across all floating point number scores between 0 and 1.
        # temp['avg_answer_score'] = (temp.answer_relevancy + temp.answer_similarity + temp.answer_correctness) / 3
        temp['avg_answer_score'] = temp.answer_correctness
        avg_answer_score = np.round(temp.avg_answer_score.mean(),4)
        temp_score = avg_answer_score
    print(f"avg_score: {temp_score}")


    # Append temp to the list of results.
    ragas_results_df_list.append(temp)
        
    

    # Return concantenated results and scores.
    ragas_results_df = pd.concat(ragas_results_df_list, ignore_index=True)
    return ragas_results_df_list
    '''
    return ragas_results

In [7]:
# Execute the evaluation.
print(f"Evaluating {EVALUATE_WHAT} using eval questions:")
ragas_result= evaluate_ragas_model(
    dataset, 
    eval_metrics, 
    what_to_evaluate=EVALUATE_WHAT)

Evaluating CONTEXTS using eval questions:


Evaluating:   0%|          | 0/2 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/2 [01:20<?, ?it/s]


TimeoutError: 

In [7]:
ragas_result

{'context_recall': nan}

In [ ]:
loop = asyncio.get_running_loop()
await loop.run_in_executor(None, self.generate, prompt)

In [6]:
# Execute the evaluation.
print(f"Evaluating {EVALUATE_WHAT} using eval questions:")
ragas_result= evaluate_ragas_model(
    dataset, 
    eval_metrics, 
    what_to_evaluate=EVALUATE_WHAT)

Evaluating CONTEXTS using eval questions:


Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/4 [02:09<?, ?it/s]
Exception in thread Thread-6:
Traceback (most recent call last):
  File "c:\app\BaseToolkit\lib\site-packages\aiohttp\connector.py", line 1025, in _wrap_create_connection
    return await self._loop.create_connection(*args, **kwargs)
  File "c:\app\BaseToolkit\lib\asyncio\base_events.py", line 1065, in create_connection
    raise exceptions[0]
  File "c:\app\BaseToolkit\lib\asyncio\base_events.py", line 1050, in create_connection
    sock = await self._connect_sock(
  File "c:\app\BaseToolkit\lib\asyncio\base_events.py", line 961, in _connect_sock
    await self.sock_connect(sock, address)
  File "c:\app\BaseToolkit\lib\asyncio\selector_events.py", line 500, in sock_connect
    return await fut
  File "c:\app\BaseToolkit\lib\asyncio\futures.py", line 284, in __await__
    yield self  # This tells Task to wait for completion.
  File "c:\app\BaseToolkit\lib\asyncio\tasks.py", line 328, in __wakeup
    future.result()
  File "c:\app\BaseTo

ExceptionInRunner: The runner thread which was running the jobs raised an exeception. Read the traceback above to debug it. You can also pass `raise_exceptions=False` incase you want to show only a warning message instead.

In [7]:
ragas_result

NameError: name 'ragas_result' is not defined

# from new documentation

## Using DeepEval

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from deepeval.models.base_model import DeepEvalBaseLLM

class Mistral7B(DeepEvalBaseLLM):
    def __init__(
        self,
        model,
        tokenizer
    ):
        self.model = model
        self.tokenizer = tokenizer

    def load_model(self):
        return self.model

    def generate(self, prompt: str) -> str:
        model = self.load_model()

        device = "cpu" # "cuda" # the device to load the model onto

        model_inputs = self.tokenizer([prompt], return_tensors="pt").to(device)
        model.to(device)

        generated_ids = model.generate(**model_inputs, max_new_tokens=100, do_sample=True)
        return self.tokenizer.batch_decode(generated_ids)[0]

    async def a_generate(self, prompt: str) -> str:
        return self.generate(prompt)

    def get_model_name(self):
        return "Mistral 7B"

model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-v0.1")
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")

mistral_7b = Mistral7B(model=model, tokenizer=tokenizer)
#print(mistral_7b.generate("Write me a joke"))


c:\app\BaseToolkit\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'deepeval'

In [ ]:
from deepeval import evaluate
from deepeval.metrics import AnswerRelevancyMetric
from deepeval.test_case import LLMTestCase

# Replace this with the actual output from your LLM application
actual_output = "We offer a 30-day full refund at no extra cost."

metric = AnswerRelevancyMetric(
    threshold=0.7,
    model="mistral_7b",
    include_reason=True
)
test_case = LLMTestCase(
    input="What if these shoes don't fit?",
    actual_output=actual_output
)

metric.measure(test_case)
print(metric.score)
print(metric.reason)

# or evaluate test cases in bulk
evaluate([test_case], [metric])

In [ ]:
from deepeval.test_case import LLMTestCase

test_case = LLMTestCase(
    input="What if these shoes don't fit?",
    expected_output="You're eligible for a 30 day refund at no extra cost.",
    actual_output="We offer a 30-day full refund at no extra cost.",
    context=["All customers are eligible for a 30 day full refund at no extra cost."],
    retrieval_context=["Only shoes can be refunded."],
    tools_used=["WebSearch"],
    expected_tools=["WebSearch", "QueryDatabase"]
)